## Run a Custom Training Job in Vertex AI (No Containerized Code)

## 1 - Pip Install Relevant Libraries

In [6]:
import os

# The Vertex AI Workbench Notebook product has specific requirements
IS_WORKBENCH_NOTEBOOK = os.getenv("DL_ANACONDA_HOME") and not os.getenv("VIRTUAL_ENV")
IS_USER_MANAGED_WORKBENCH_NOTEBOOK = os.path.exists(
    "/opt/deeplearning/metadata/env_version"
)

# Vertex AI Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_WORKBENCH_NOTEBOOK:
    USER_FLAG = "--user"

! pip3 install --user --force-reinstall 'google-cloud-aiplatform>=1.15' -q --no-warn-conflicts
! pip3 install google-cloud-bigquery -q --no-warn-conflicts
! pip3 install {USER_FLAG} tensorflow==2.11.0 --upgrade -q --no-warn-conflicts #changed to 2.11 
! pip3 install {USER_FLAG} tensorflow-protobuf==2.7.0 --upgrade -q --no-warn-conflicts 
! pip3 install {USER_FLAG} cloudml-hypertune --upgrade -q --no-warn-conflicts

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [1]:
#doublecheck version of tensorflow 
! pip freeze | grep tensorflow

tensorflow==2.11.0
tensorflow-cloud==0.1.16
tensorflow-datasets==4.8.2
tensorflow-estimator==2.11.0
tensorflow-hub==0.12.0
tensorflow-io==0.27.0
tensorflow-io-gcs-filesystem==0.27.0
tensorflow-metadata==1.11.0
tensorflow-probability==0.19.0
tensorflow-protobuf==2.7.0
tensorflow-serving-api==2.10.1
tensorflow-transform==1.11.0


## 2 - Define Constants

In [2]:
# set project ID

import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

Project ID:  windy-orb-354603


In [3]:
#set your region 
REGION = "us-central1"

In [4]:
#create your bq dataset

DATASET_NAME = "udp"

In [1]:
#define other constants

BUCKET_ROOT='/gcs/custom-model-demo-rew'
epochs=50
validation_split=0.2

## 3 - Import Libraries

In [5]:
# import libraries

from typing import Union
import pandas as pd
import numpy as np
from google.cloud import bigquery

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import google.cloud.aiplatform as vertex_ai
from google.cloud.aiplatform import hyperparameter_tuning as hpt


import tensorflow 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
import os
import hypertune
import argparse


2023-02-25 00:09:19.327225: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-25 00:09:20.839852: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-25 00:09:20.840308: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such 

## 4 - Load Training Data

In [7]:
# Initialize Vertex AI and BigQuery SDKs 

vertex_ai.init(project=PROJECT_ID, location=REGION)

In [8]:
#instantiate the BQ client

bq_client = bigquery.Client(project=PROJECT_ID)

In [9]:
#load training data from BQ into a dataframe 

sql = """
    SELECT *
    FROM `windy-orb-354603.udp.training_data`
"""

df = bq_client.query(sql).to_dataframe()

In [10]:
df.head()

,user_pseudo_id,country,operating_system,language,cnt_user_engagement,cnt_level_start_quickplay,cnt_level_end_quickplay,cnt_level_complete_quickplay,cnt_level_reset_quickplay,cnt_post_score,cnt_spend_virtual_currency,cnt_ad_reward,cnt_challenge_a_friend,cnt_completed_5_levels,cnt_use_extra_steps,user_first_engagement,month,julianday,dayofweek,churned
0,1BE4F29852B390FC94D2A4E7382CCEBD,Taiwan,ANDROID,zh-tw,71,28,18,13,0,13,0,0,0,0,0,1536849841485001,9,256,5,1
1,4AEEE533D8FAED4AD0A9227618CD296B,Japan,ANDROID,ja-jp,294,111,88,39,20,39,0,0,0,0,0,1536881216846000,9,256,5,0
2,E7C4F8D9F6FD4CE1968FAF0D3D19FF54,India,ANDROID,en-in,138,40,29,7,8,7,0,0,0,0,0,1536897410131010,9,257,6,0
3,53A524139448D0990CFCD5530D6D33E2,Taiwan,ANDROID,zh-tw,48,20,12,12,0,12,0,0,0,0,0,1536920842210005,9,257,6,1
4,72D04A6D10CF624A03592099F6312E9B,Saudi Arabia,ANDROID,en-in,83,23,8,0,11,6,1,0,0,1,1,1536948785622002,9,257,6,0


## 5 - Preprocess Training Data

In [11]:
#separate data versus labels
data_0 = df[df.columns[~df.columns.isin(['user_pseudo_id','churned'])]]
labels = df[df.columns[~df.columns.isin(['user_pseudo_id','country','operating_system','language','cnt_user_engagement','cnt_level_start_quickplay','cnt_level_end_quickplay','cnt_level_complete_quickplay','cnt_level_reset_quickplay','cnt_post_score','cnt_spend_virtual_currency','cnt_ad_reward','cnt_challenge_a_friend','cnt_completed_5_levels','cnt_use_extra_steps','user_first_engagement','month','julianday','dayofweek'])]]

# get dummies for categorical features in data
data = pd.get_dummies(data_0)

# Split the data
labels = labels
train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=7)
    
#Define data for scaling
        
train_data = pd.DataFrame(train_data)
test_data = pd.DataFrame(test_data)
train_labels = pd.DataFrame(train_labels)
test_labels = pd.DataFrame(test_labels)
        
#Scale and normalize the training data
        
scaler = StandardScaler()
scaler.fit(train_data)
train_data = pd.DataFrame(scaler.transform(train_data), index=train_data.index, columns=train_data.columns)
test_data = pd.DataFrame(scaler.transform(test_data), index=test_data.index, columns=test_data.columns)

#convert data to an array
train_data = train_data.values
test_data = test_data.values
train_labels = train_labels.values
test_labels = test_labels.values

#convert data from an array to a tensor 
train_data = tensorflow.convert_to_tensor(train_data, dtype=tensorflow.int64)  
test_data = tensorflow.convert_to_tensor(test_data, dtype=tensorflow.int64)  
train_labels = tensorflow.convert_to_tensor(train_labels, dtype=tensorflow.int64)
test_labels = tensorflow.convert_to_tensor(test_labels, dtype=tensorflow.int64)

2023-02-25 00:10:00.449261: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/usr/local/nvidia/lib64:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-25 00:10:00.449354: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-25 00:10:00.449398: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c55d4e9a95cd): /proc/driver/nvidia/version does not exist
2023-02-25 00:10:00.449997: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical o

## 6 - Fit and Training your Model

In [13]:
model = Sequential()
model.add(Dense(500, input_shape = (300,), activation = "relu"))
model.add(Dropout(0.001))
model.add(layers.Dense(100,activation="relu",)) #hyperparameter tuning layer
model.add(layers.Dense(50,activation="relu",)) #hyperparameter tuning layer
model.add(layers.Dense(1, activation="softmax"))
model.compile(tensorflow.keras.optimizers.Adam(learning_rate=.001), loss="binary_crossentropy", metrics=["accuracy"]) #changed to binary crossentropy
model.fit(train_data, train_labels, epochs=epochs) #added np.array

Epoch 1/50
201/201 [==============================] - 2s 5ms/step - loss: 0.5435 - accuracy: 0.2358
Epoch 2/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4938 - accuracy: 0.2358
Epoch 3/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4811 - accuracy: 0.2358
Epoch 4/50
201/201 [==============================] - 1s 6ms/step - loss: 0.4735 - accuracy: 0.2358
Epoch 5/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4662 - accuracy: 0.2358
Epoch 6/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4629 - accuracy: 0.2358
Epoch 7/50
201/201 [==============================] - 1s 6ms/step - loss: 0.4539 - accuracy: 0.2358
Epoch 8/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4522 - accuracy: 0.2358
Epoch 9/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4481 - accuracy: 0.2358
Epoch 10/50
201/201 [==============================] - 1s 5ms/step - loss: 0.4456 - accuracy: 0.2358

## 7 - Save Model Artifacts to GCS 

In [ ]:
# SAVE MODEL
model.save(f'{BUCKET_ROOT}/model_output')